### Variables

In [ ]:
host = ''
port = "5432"
dbname = "db_demo_rh"
user = ""
password = ""
schema = ""

### Functions

In [ ]:
import psycopg2
import os

def connect_db(host, port, dbname, user, password):
    try:
        connection = psycopg2.connect(
            host=host,
            port=port,
            dbname=dbname,
            user=user,
            password=password
        )
        return connection
    except Exception as e:
        print(f"Error in conn to db: {e}")
        return None

def get_db_structure(connection, schema):
    try:
        cursor = connection.cursor()
        cursor.execute(f"""
            SELECT
                table_name,
                column_name,
                udt_name,
                character_maximum_length,
                is_nullable,
                column_default
            FROM
                information_schema.columns
            WHERE
                table_schema = '{schema}' AND table_name NOT IN (
                    SELECT table_name
                    FROM information_schema.views
                    WHERE table_schema = '{schema}'
                )
            ORDER BY
                table_name, ordinal_position
        """)
        columns = cursor.fetchall()

        cursor.execute(f"""
            SELECT
                kcu.table_name,
                tco.constraint_type,
                kcu.column_name,
                ccu.table_name AS foreign_table_name,
                ccu.column_name AS foreign_column_name
            FROM
                information_schema.table_constraints tco
            JOIN
                information_schema.key_column_usage kcu 
            ON
                tco.constraint_name = kcu.constraint_name
            JOIN
                information_schema.constraint_column_usage ccu 
            ON
                ccu.constraint_name = tco.constraint_name
            WHERE
                tco.constraint_type IN ('PRIMARY KEY', 'FOREIGN KEY')
                AND tco.table_schema = '{schema}'
            ORDER BY
                kcu.table_name, kcu.ordinal_position
        """)
        constraints = cursor.fetchall()

        cursor.execute(f"""
            SELECT
                proname,
                pg_get_functiondef(p.oid) AS function_definition
            FROM
                pg_proc p
            JOIN
                pg_namespace n ON n.oid = p.pronamespace
            WHERE
                n.nspname = '{schema}'
        """)
        functions = cursor.fetchall()

        cursor.execute(f"""
            SELECT
                table_name,
                view_definition
            FROM
                information_schema.views
            WHERE
                table_schema = '{schema}'
        """)
        views = cursor.fetchall()

        return columns, constraints, functions, views
    except Exception as e:
        print(f"Error gathering db structuree: {e}")
        return None, None, None, None


def generate_sql_script(columns, constraints, functions, views, schema):
    tables = {}
    primary_keys = {}
    foreign_keys = {}

    if columns is not None and constraints is not None:

        for table, column, udt_name, char_length, is_nullable, column_default in columns:
            if table not in tables:
                tables[table] = []
            col_type = udt_name
            if udt_name == 'varchar' and char_length is not None:
                col_type = f"varchar({char_length})"
            col_def = f"{column} {col_type}"
            if is_nullable == 'NO':
                col_def += " NOT NULL"
            if column_default:
                col_def += f" DEFAULT {column_default}"
            tables[table].append(col_def)

        for table, constraint_type, column_name, foreign_table, foreign_column in constraints:
            if constraint_type == 'PRIMARY KEY':
                if table not in primary_keys:
                    primary_keys[table] = []
                primary_keys[table].append(column_name)
            elif constraint_type == 'FOREIGN KEY':
                if table not in foreign_keys:
                    foreign_keys[table] = []
                foreign_keys[table].append((column_name, foreign_table, foreign_column))

        sql_script = "-- SQL Script to create tables\n\n"

        for table, cols in tables.items():
            sql_script += f"CREATE TABLE IF NOT EXISTS {schema}.{table} (\n"
            sql_script += ",\n".join(f"    {col}" for col in cols)
            if table in primary_keys:
                pk = ", ".join(primary_keys[table])
                sql_script += f",\n    PRIMARY KEY ({pk})"
            if table in foreign_keys:
                for col, foreign_table, foreign_column in foreign_keys[table]:
                    sql_script += f",\n    FOREIGN KEY ({col}) REFERENCES {foreign_table}({foreign_column})"
            sql_script += "\n);\n\n"

    if functions is not None:
        sql_script += "-- SQL Script to create functions\n\n"

        for function_name, function_definition in functions:
            sql_script += f"{function_definition}\n\n"

    if views is not None:
        sql_script += "-- SQL Script to create views\n\n"

        for view_name, view_definition in views:
            sql_script += f"CREATE OR REPLACE VIEW {schema}.{view_name} AS\n{view_definition}\n\n"

    return sql_script


def export_to_sql(file_path, sql_script):
    try:
        with open(file_path, 'w+') as f:
            f.write(sql_script)
        
        print(sql_script)
        print(f"\nMigration script exported to {file_path}")
    except Exception as e:
        print(f"Error exporting SQL file: {e}")


### Execute

In [ ]:
connection = connect_db(host, port, dbname, user, password)

if connection:
    
    columns, constraints, functions, views = get_db_structure(connection, schema)

    sql_script = generate_sql_script(columns, constraints, functions, views, schema)

    
    file_path = os.path.join(os.getcwd(), "migration.sql")

    export_to_sql(file_path, sql_script)

    
    connection.close()